In [72]:
import pandas as pd
import numpy as np
import math
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import gc

import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter

# Import libraries
import nltk.translate.chrf_score as chrf
from rouge import Rouge
import nltk
import nltk.translate.gleu_score as gleu

import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')
import nltk.translate.meteor_score as meteor
import nltk.translate.nist_score as nist
import nltk.translate.bleu_score as bleu

#import spacy
from nltk.tokenize import RegexpTokenizer

from sklearn.metrics.pairwise import cosine_similarity
#import itertools

import codecs
import jieba
#Import nltk Snowball stemmer
from nltk.stem.snowball import SnowballStemmer

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from stop_words import get_stop_words
import transformers
from transformers import BertTokenizer, TFBertModel
import bert_score as bert


#Import the Voikko to work with finnish lemmas
import libvoikko

In [426]:
scores_cs = pd.read_csv("corpus/cs-en/scores.csv")
scores_de = pd.read_csv("corpus/de-en/scores.csv")
scores_en_fi = pd.read_csv("corpus/en-fi/scores.csv")
scores_en_zh = pd.read_csv("corpus/en-zh/scores.csv")
scores_ru = pd.read_csv("corpus/ru-en/scores.csv")
scores_zh = pd.read_csv("corpus/zh-en/scores.csv")

In [4]:
# Let's check all csv files
scores_cs.head(1)

,source,reference,translation,z-score,avg-score,annotators
0,Uchopíte pak zbraň mezi své předloktí a rameno...,You will then grab the weapon between your for...,You then grasp the gun between your forearm an...,-0.675383,60.0,3


In [5]:
scores_de.head(1)

,source,reference,translation,z-score,avg-score,annotators
0,"Ihr Zeitlupentempo maßen sie, als sie vor Spit...",Her timeless pace measures them when they equi...,Their slow speed was measured by researchers o...,-0.345024,76.0,1


In [6]:
scores_en_fi.head(1)

,source,reference,translation,z-score,avg-score,annotators
0,"You can turn yourself into a pineapple, a dog ...","Voit muuttaa itsesi ananasta, koirasta tai Roy...","Voit muuttaa itsesi ananakseksi, koiraksi tai ...",-0.286195,34.2,5


In [7]:
scores_en_zh.head(1)

,source,reference,translation,z-score,avg-score,annotators
0,"""In the GISS model's simulation, Venus' slow s...",GSIS的科学家AnthonyDelGenio在新闻稿中解释说：“在GISS模型的模拟模型中...,戈达德太空研究所科学家安东尼·德尔·杰尼奥在新闻发布会上解释说：“在戈达德太空研究所的模型模...,-1.171867,50.0,1


In [8]:
scores_ru.head(1)

,source,reference,translation,z-score,avg-score,annotators
0,В этом году крымчане получат уведомление на оп...,This year the Crimeans will receive a notice f...,"This year, residents of Crimea will receive a ...",0.878043,92.0,1


In [9]:
scores_zh.head(1)

,source,reference,translation,z-score,avg-score,annotators
0,他性格活泼，这对英国赛马来说是好事，但是除此之外，他还是一位不可思议的骑师。,"His character is good for the British horse, b...",He's a lively character which is good for Brit...,0.625559,92.75,4


In [384]:
# Work with contractions using pycontractions lib that returns a proper form for
# shortened version of the spoken and written forms 
# without ambiguous "you'd" -> "you had / you would"

from pycontractions import Contractions

# Download 'GoogleNews-vectors-negative300.bin' file to a folder with this jupiter notebook
# if needed (size is more 3 Gb)
# archive on - https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
cont = Contractions('GoogleNews-vectors-negative300.bin')

# prevents loading on first expand_texts call
# usually takes time
cont.load_models()

In [11]:
# Check how pycontractions works:
a = scores_de.translation
a[6]

"It's now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work."

In [12]:
b = pd.Series(list(cont.expand_texts(a, precise=True)))

In [13]:
# Check how contracion expanding worked
b[6]

'it is now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.'

In [32]:
# As contraction extractions work really slow
# they were done outside preprocessing function
references_cs = pd.Series(list(cont.expand_texts(scores_cs['reference'], precise=True)))
references_de = pd.Series(list(cont.expand_texts(scores_de['reference'], precise=True)))
references_ru = pd.Series(list(cont.expand_texts(scores_ru['reference'], precise=True)))
references_zh = pd.Series(list(cont.expand_texts(scores_zh['reference'], precise=True)))

In [33]:
# Same for translations
translation_cs = pd.Series(list(cont.expand_texts(scores_cs['translation'], precise=True)))
translation_de = pd.Series(list(cont.expand_texts(scores_de['translation'], precise=True)))
translation_ru = pd.Series(list(cont.expand_texts(scores_ru['translation'], precise=True)))
translation_zh = pd.Series(list(cont.expand_texts(scores_zh['translation'], precise=True)))

In [16]:
# Check how expanding contractions worked
print(scores_de['translation'][6], '   ---->  \n', translation_de[6])

It's now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.    ---->  
 it is now time to look at the needs of community legal centres and their client communities, and for the Palaszczuk government to invest in this important work.


In [383]:
# release references and 'garbage collect' to free memory
import gc
del cont
gc.collect()

8072

In [350]:
def preprocessing(series, lang, pontuation = False, lemmatize=False, stemmer=False, stop_words = False):
    processed_corpus = []

    for i in tqdm(range(len(series))):
        text = series[i]
        
        #LOWERCASE
        text = text.lower()
        
        #REMOVE TAGS
        text = BeautifulSoup(text).get_text()
        
        #REMOVE PONTUATION
        if pontuation==True:
            text = re.sub(r'[\s+\.\!\/_,$%^*(+\"\']+|[+——！，？、~@#￥%……&*（）:：；《）《》“”()»〔〕-]+', ' ', text)
           
        #LEMMATIZATION
        if lemmatize==True:
            if lang == 'eng':
                lemma = WordNetLemmatizer()
                text = ' '.join([lemma.lemmatize(word) for word in text.split()])
            elif lang == 'finnish':
                temp = []
                v = libvoikko.Voikko(u"fi")
                for word in text.split():
                    voikko_dict = v.analyze(word)
                    try:
                        t = voikko_dict[0]['BASEFORM']
                        temp.append(t)
                    except:
                        temp.append(word)
                        continue
                text = temp.copy()
                text = ' '.join(text)
                text = text.lower()
            else:
                pass
        #STEMMER
        if stemmer==True:
            if lang == 'finnish':
                snowball_stemmer = SnowballStemmer('finnish')
                text = ' '.join([snowball_stemmer.stem(word) for word in text.split()])
                
            elif lang == 'eng':
                snowball_stemmer = SnowballStemmer('english')
                text = ' '.join([snowball_stemmer.stem(word) for word in text.split()])
            else:
                pass
        
        
        #REMOVE STOP WORDS
        if stop_words==True:
            if lang == 'finnish':
                stop_words = get_stop_words('finnish')
                text = ' '.join([word for word in text.split() if word not in stop_words])
            elif lang == 'zh':
                stop_words = set(line.strip() for line in open('stopwords-zh.txt', encoding='utf8'))
                text = list(jieba.cut_for_search(text))
                text = ' '.join([word for word in text if word not in stop_words])
            else:
                stop_words = set(stopwords.words('english')).union(STOP_WORDS)
                text = ' '.join([word for word in text.split() if word not in stop_words])
        
        # For en-zh language pair use jieba for words cut
        if lang == 'zh':
            text = list(jieba.cut_for_search(text))
        else:
            # Convert to list from string
            text = text.split()   

        text = " ".join(text)
        processed_corpus.append(text)
    return processed_corpus

In [253]:
def prep(ref_trans, pontuation = True, lemmatize=True, stemmer=True, stop_words = True):
    
    if ref_trans == 'referece':
        a,b,c = references_cs, references_de, scores_en_fi['reference']
        d,e,f = scores_en_zh['reference'], references_ru, references_zh
    elif ref_trans == 'translation':
        a,b,c = translation_cs, translation_de, scores_en_fi['translation']
        d,e,f = scores_en_zh['translation'], translation_ru, translation_zh
        
    ref_trans_cs = preprocessing(a,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_de = preprocessing(b,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_en_fi = preprocessing(c,'finnish', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_en_zh = preprocessing(d,'zh', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_ru = preprocessing(e,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_zh = preprocessing(f,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    out = ref_trans_cs, ref_trans_de, ref_trans_en_fi, ref_trans_en_zh, ref_trans_ru, ref_trans_zh
    
    return out

In [535]:
# All parameters set to False

references_cs1, references_de1, references_en_fi1, references_en_zh1, references_ru1, references_zh1 = prep(
    'referece', pontuation = False, lemmatize=False, stemmer=False, stop_words = False)

translation_cs1, translation_de1, translation_en_fi1, translation_en_zh1, translation_ru1, translation_zh1 = prep(
    'translation', pontuation = False, lemmatize=False, stemmer=False, stop_words = False)

In [218]:
# Create functions for evaluate ready metrics for all reference/translation pairs
def chrf_metric(translation, references):
    chrf_metric = []
    for i in range(len(translation)):
        row = chrf.sentence_chrf([translation[i]], references[i])
        chrf_metric.append(row)
    return chrf_metric

def gleu_metric(translation, references):
    gleu_metric = []
    for i in range(len(translation)):
        row = gleu.sentence_gleu([str(references[i]).split()], str(translation[i]).split(), min_len=1, max_len=2)
        gleu_metric.append(row)
    return gleu_metric
def meteor_metric(translation, references):
    meteor_metric = []
    for i in range(len(translation)):
        row = (meteor.meteor_score([translation[i]], references[i]))
        meteor_metric.append(row)
    return meteor_metric
def bleu_metric(translation, references):
    bleu_metric = []
    for i in range(len(translation)):
        row = (bleu.sentence_bleu([translation[i]], references[i]))
        bleu_metric.append(row)
    return bleu_metric

def nist_metric(translation, references):
    nist_metric = []
    for i in range(len(translation)):
        row = (nist.sentence_nist([translation[i]], references[i], n=1))
        nist_metric.append(row)
    return nist_metric
def bert_metric(translation, references, lang):
    P, R, bert_metric = bert.score(translation, references, lang=lang, verbose=True)
    return bert_metric

In [236]:
def corr(scores, b = False):
    chf = scores['z-score'].corr(scores['chf'])
    gleu = scores['z-score'].corr(scores['gleu'])
    meteor = scores['z-score'].corr(scores['meteor'])
    bleu = scores['z-score'].corr(scores['bleu'])
   # nist = scores['z-score'].corr(scores['nist'])
    if b == True:
        bert = scores['z-score'].corr(scores['bert'])
        corr = [chf, gleu, meteor, bleu, bert]
    else:
        bert = 0
        corr = [chf, gleu, meteor, bleu]
   
    return corr

In [237]:
def create_table(t1,t2,t3,t4,t5,t6, r1,r2,r3,r4,r5,r6, b = False):
    chf_cs = chrf_metric(t1, r1)
    scores_cs['chf'] = chf_cs
    chf_de = chrf_metric(t2, r2)
    scores_de['chf'] = chf_de
    chf_en_fi = chrf_metric(t3, r3)
    scores_en_fi['chf'] = chf_en_fi
    chf_en_zh = chrf_metric(t4, r4)
    scores_en_zh['chf'] = chf_en_zh
    chf_ru = chrf_metric(t5, r5)
    scores_ru['chf'] = chf_ru
    chf_zh = chrf_metric(t6, r6)
    scores_zh['chf'] = chf_zh
    
    gleu_cs = gleu_metric(t1, r1)
    scores_cs['gleu'] = gleu_cs
    gleu_de = gleu_metric(t2, r2)
    scores_de['gleu'] = gleu_de
    gleu_en_fi = gleu_metric(t3, r3)
    scores_en_fi['gleu'] = gleu_en_fi
    gleu_en_zh = gleu_metric(t4, r4)
    scores_en_zh['gleu'] = gleu_en_zh
    gleu_ru = gleu_metric(t5, r5)
    scores_ru['gleu'] = gleu_ru
    gleu_zh = gleu_metric(t6, r6)
    scores_zh['gleu'] = gleu_zh
    
    meteor_cs = meteor_metric(t1, r1)
    scores_cs['meteor'] = meteor_cs
    meteor_de = meteor_metric(t2, r2)
    scores_de['meteor'] = meteor_de
    meteor_en_fi = meteor_metric(t3, r3)
    scores_en_fi['meteor'] = meteor_en_fi
    meteor_en_zh = meteor_metric(t4, r4)
    scores_en_zh['meteor'] = meteor_en_zh
    meteor_ru = meteor_metric(translation_ru, references_ru)
    scores_ru['meteor'] = meteor_ru
    meteor_zh = meteor_metric(translation_zh, references_zh)
    scores_zh['meteor'] = meteor_zh
    
    bleu_cs = bleu_metric(t1, r1)
    scores_cs['bleu'] = bleu_cs
    bleu_de = bleu_metric(t2, r2)
    scores_de['bleu'] = bleu_de
    bleu_en_fi = bleu_metric(t3, r3)
    scores_en_fi['bleu'] = bleu_en_fi
    bleu_en_zh = bleu_metric(t4, r4)
    scores_en_zh['bleu'] = bleu_en_zh
    bleu_ru = bleu_metric(t5, r5)
    scores_ru['bleu'] = bleu_ru
    bleu_zh = bleu_metric(t6, r6)
    scores_zh['bleu'] = bleu_zh
    
#     nist_cs = nist_metric(t1, r1)
#     scores_cs['nist'] = nist_cs
#     nist_de = nist_metric(t2, r2)
#     scores_de['nist'] = nist_de
#     nist_en_fi = nist_metric(t3, r3)
#     scores_en_fi['nist'] = nist_en_fi
#     nist_en_zh = nist_metric(t4, r4)
#     scores_en_zh['nist'] = nist_en_zh
#     nist_ru = nist_metric(t5, r5)
#     scores_ru['nist'] = nist_ru
#     nist_zh = nist_metric(t6, r6)
#     scores_zh['nist'] = nist_zh
    
    if b == True:
        bert_cs = bert_metric(t1, r1, 'en')
        scores_cs['bert'] = bert_cs
        bert_de = bert_metric(t2, r2, 'en')
        scores_de['bert'] = bert_de
        bert_en_fi = bert_metric(t3, r3, 'others')
        scores_en_fi['bert'] = bert_en_fi
        bert_en_zh = bert_metric(t4, r4, 'zh')
        scores_en_zh['bert'] = bert_en_zh
        bert_ru = bert_metric(t5, r5, 'en')
        scores_ru['bert'] = bert_ru
        bert_zh = bert_metric(t6, r6, 'en')
        scores_zh['bert'] = bert_zh
        
        columns = ['chrf', 'gleu', 'meteor', 'bleu', 'bert']
        
    else:
        columns = ['chrf', 'gleu', 'meteor', 'bleu']

    corr_cs = corr(scores_cs, b = b)
    corr_de = corr(scores_de, b = b)
    corr_en_fi = corr(scores_en_fi, b =b)
    corr_en_zh = corr(scores_en_zh, b = b)
    corr_ru = corr(scores_ru, b = b)
    corr_zh = corr(scores_zh, b = b)
    
    data = []
    index = ['scores_cs', 'scores_de', 'scores_en_fi', 'scores_en_zh', 'scores_ru', 'scores_zh']
    correlation = pd.DataFrame(data, columns=columns, index = index)
    correlation.iloc[0] = corr_cs
    correlation.iloc[1] = corr_de
    correlation.iloc[2] = corr_en_fi
    correlation.iloc[3] = corr_en_zh
    correlation.iloc[4] = corr_ru
    correlation.iloc[5] = corr_zh
    return correlation

In [537]:
# Calculate baseline results for all ready metrics
create_table(translation_cs1, translation_de1, translation_en_fi1, translation_en_zh1, translation_ru1, translation_zh1,
             references_cs1, references_de1, references_en_fi1, references_en_zh1, references_ru1, references_zh1,
             b = False)

,chrf,gleu,meteor,bleu
scores_cs,0.462239,0.427909,0.439981,0.468781
scores_de,0.341172,0.310139,0.308153,0.346757
scores_en_fi,0.611567,0.494636,0.491475,0.619928
scores_en_zh,0.423398,0.449157,0.453092,0.468428
scores_ru,0.361388,0.333465,0.336711,0.367557
scores_zh,0.341228,0.317931,0.326447,0.351904


In [255]:
# Punctuation == True

references_cs2, references_de2, references_en_fi2, references_en_zh2,references_ru2, references_zh2 = prep(
    'referece', pontuation = True, lemmatize=False, stemmer=False, stop_words = False)

translation_cs2, translation_de2, translation_en_fi2, translation_en_zh2, translation_ru2, translation_zh2 = prep(
    'translation', pontuation = True, lemmatize=False, stemmer=False, stop_words = False)

In [256]:
# Calculate results for all ready metrics with deletion of Punctuation
create_table(translation_cs2, translation_de2, translation_en_fi2, translation_en_zh2, translation_ru2, translation_zh2,
             references_cs2, references_de2, references_en_fi2, references_en_zh2, references_ru2, references_zh2,
             b = False)

,chrf,gleu,meteor,bleu
scores_cs,0.460175,0.442928,0.459244,0.46626
scores_de,0.339221,0.323857,0.323407,0.343768
scores_en_fi,0.606508,0.521099,0.516653,0.616161
scores_en_zh,0.419761,0.44232,0.451714,0.475039
scores_ru,0.35807,0.341896,0.336711,0.362122
scores_zh,0.337752,0.329073,0.326447,0.347637


In [257]:
# Lemmatize == True

references_cs3, references_de3, references_en_fi3, references_en_zh3,references_ru3, references_zh3 = prep(
    'referece', pontuation = False, lemmatize=True, stemmer=False, stop_words = False)

translation_cs3, translation_de3, translation_en_fi3, translation_en_zh3, translation_ru3, translation_zh3 = prep(
    'translation', pontuation = False, lemmatize=True, stemmer=False, stop_words = False)

In [258]:
# Re-calculate all ready metrics with Lemmatize == True
create_table(translation_cs3, translation_de3, translation_en_fi3, translation_en_zh3, translation_ru3, translation_zh3,
             references_cs3, references_de3, references_en_fi3, references_en_zh3, references_ru3, references_zh3,
             b = False)

,chrf,gleu,meteor,bleu
scores_cs,0.461182,0.42928,0.44041,0.468664
scores_de,0.339801,0.311003,0.308517,0.345696
scores_en_fi,0.596299,0.525408,0.521339,0.60642
scores_en_zh,0.423398,0.449157,0.453092,0.468428
scores_ru,0.361224,0.33507,0.336711,0.366839
scores_zh,0.340611,0.318608,0.326447,0.350394


In [259]:
# Stemmer == True

references_cs4, references_de4, references_en_fi4, references_en_zh4,references_ru4, references_zh4 = prep(
    'referece', pontuation = False, lemmatize=False, stemmer=True, stop_words = False)

translation_cs4, translation_de4, translation_en_fi4, translation_en_zh4, translation_ru4, translation_zh4 = prep(
    'translation', pontuation = False, lemmatize=False, stemmer=True, stop_words = False)

In [260]:
# Re-calculate all ready metrics with Stemmer == True
create_table(translation_cs4, translation_de4, translation_en_fi4, translation_en_zh4, translation_ru4, translation_zh4,
             references_cs4, references_de4, references_en_fi4, references_en_zh4, references_ru4, references_zh4,
             b = False)

,chrf,gleu,meteor,bleu
scores_cs,0.458551,0.433943,0.440132,0.466033
scores_de,0.33548,0.312284,0.310658,0.342397
scores_en_fi,0.595025,0.510169,0.506616,0.609784
scores_en_zh,0.423398,0.449157,0.453092,0.468428
scores_ru,0.360056,0.338979,0.336711,0.366964
scores_zh,0.342474,0.322487,0.326447,0.353436


In [261]:
# Stop_words == True

references_cs5, references_de5, references_en_fi5, references_en_zh5, references_ru5, references_zh5 = prep(
    'referece', pontuation = False, lemmatize=False, stemmer=False, stop_words = True)

translation_cs5, translation_de5, translation_en_fi5, translation_en_zh5, translation_ru5, translation_zh5 = prep(
    'translation', pontuation = False, lemmatize=False, stemmer=False, stop_words = True)

In [262]:
# Re-calculate all ready metrics with stop_words == True
create_table(translation_cs5, translation_de5, translation_en_fi5, translation_en_zh5, translation_ru5, translation_zh5,
             references_cs5, references_de5, references_en_fi5, references_en_zh5, references_ru5, references_zh5,
             b = False)

,chrf,gleu,meteor,bleu
scores_cs,0.462193,0.427878,0.439901,0.468756
scores_de,0.34117,0.310134,0.308149,0.346754
scores_en_fi,0.611555,0.494621,0.49147,0.619918
scores_en_zh,0.423409,0.449227,0.453161,0.468476
scores_ru,0.361388,0.333425,0.336711,0.36755
scores_zh,0.341249,0.317942,0.326447,0.351916


In [265]:
# All parameters of pre-processing set to  True except Stemmer

references_cs6, references_de6, references_en_fi6, references_en_zh6, references_ru6, references_zh6 = prep(
    'referece', pontuation = True, lemmatize=True, stemmer=False, stop_words = True)

translation_cs6, translation_de6, translation_en_fi6, translation_en_zh6, translation_ru6, translation_zh6 = prep(
    'translation', pontuation = True, lemmatize=True, stemmer=False, stop_words = True)

In [266]:
# Re-calculate all ready metrics with all parameters set to True
create_table(translation_cs6, translation_de6, translation_en_fi6, translation_en_zh6, translation_ru6, translation_zh6,
             references_cs6, references_de6, references_en_fi6, references_en_zh6, references_ru6, references_zh6,
             b = True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.



computing greedy matching.



done in 2524.76 seconds, 4.59 sentences/sec


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.



computing greedy matching.



done in 4973.61 seconds, 5.31 sentences/sec


,chrf,gleu,meteor,bleu,bert
scores_cs,0.458563,0.444786,0.460774,0.465282,0.567269
scores_de,0.337663,0.326861,0.325655,0.342394,0.421566
scores_en_fi,0.590391,0.560004,0.556457,0.603164,0.615133
scores_en_zh,0.419763,0.442372,0.451762,0.475075,0.541409
scores_ru,0.357554,0.345029,0.336711,0.360968,0.418593
scores_zh,0.3367,0.33273,0.326447,0.345333,0.416299


## Calculate cosine similarities for sentences using word embeddings
 - we use pre-trained word embeddings

In [509]:
def prep_cos(ref_trans, pontuation = True, lemmatize=True, stemmer=True, stop_words = True):
    
    if ref_trans == 'referece':
        a,b,c = scores_cs['reference'], scores_de['reference'], scores_en_fi['reference']
        d,e,f = scores_en_zh['reference'], scores_ru['reference'].astype(str), scores_zh['reference']
    elif ref_trans == 'translation':
        a,b,c = scores_cs['translation'], scores_de['translation'], scores_en_fi['translation']
        d,e,f = scores_en_zh['translation'], scores_ru['translation'].astype(str), scores_zh['translation']
        
    ref_trans_cs = preprocessing(a,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_de = preprocessing(b,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_en_fi = preprocessing(c,'finnish', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_en_zh = preprocessing(d,'zh', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_ru = preprocessing(e,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_zh = preprocessing(f,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    out = ref_trans_cs, ref_trans_de, ref_trans_en_fi, ref_trans_en_zh, ref_trans_ru, ref_trans_zh
    
    return out

In [510]:
# All parameters set to False

references_cs10, references_de10, references_en_fi10, references_en_zh10, references_ru10, references_zh10 = prep_cos(
    'referece', pontuation = False, lemmatize=False, stemmer=False, stop_words = False)

translation_cs10, translation_de10, translation_en_fi10, translation_en_zh10, translation_ru10, translation_zh10 = prep_cos(
    'translation', pontuation = False, lemmatize=False, stemmer=False, stop_words = False)

In [511]:
# All parameters of pre-processing set to  True except Stemmer

references_cs11, references_de11, references_en_fi11, references_en_zh11, references_ru11, references_zh11 = prep_cos(
    'referece', pontuation = True, lemmatize=True, stemmer=False, stop_words = True)

translation_cs11, translation_de11, translation_en_fi11, translation_en_zh11, translation_ru11, translation_zh11 = prep_cos(
    'translation', pontuation = True, lemmatize=True, stemmer=False, stop_words = True)

In [477]:
# english stopwords
english_stop_words = set(stopwords.words('english')).union(STOP_WORDS)

# chinese stopwords
#stopwords_zh = codecs.open('stopwords-zh.txt', 'r', 'utf-8').read().split(',')
stopwords_zh = set(line.strip() for line in open('stopwords-zh.txt', encoding='utf8'))

# finnish stopwords
stop_words_fi = get_stop_words('finnish')

In [478]:
# Count frequency of words

def words_count(text):
    tokenizer = RegexpTokenizer(r'\w+')
    # tokenize text
    tokens = tokenizer.tokenize(text)
   
    # delete stop words
    tokens = list(filter(lambda x: x not in english_stop_words, tokens))
    
    # lemmatize the words WordNetLemmatizer from nlkt
    # this made Pearson correlation lower - so was commented
    
    # tokens = list(map(lambda x: nltk.WordNetLemmatizer().lemmatize(x), tokens))
    
    # With spacy lemmatizer (worked better than nltk)
    # it is commented because it made correlation lower
    # tokens = list(map(lambda x: [token.lemma_ for token in nlp(x)], tokens))
    # tokens = list(itertools.chain(*tokens))
    
    return list(Counter(tokens).most_common())

In [460]:
# Count frequency of words

def words_count_zh(text):
#     # Tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    # tokenize text
    tokens = tokenizer.tokenize(text)
    # delete stop words
    tokens = list(filter(lambda x: x not in stopwords_zh, tokens))
    return list(Counter(tokens).most_common())

In [416]:
# Load the pre-trained GloVe (Global Vectors for Word Representation) word vectors.
# For cs, de, ru, zh that translated to english
# 6B tokens, 400K vocab, (300d version).
# Download file from https://www.kaggle.com/thanakomsn/glove6b300dtxt
# to a folder with this jupiter notebook (size 1Gb)
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
    return word_to_vec_map
# Download glove.6B.300d.txt to a folder with this jupiter notebook
word_to_vec_map = read_glove_vecs('glove.6B.300d.txt')

In [400]:
# Chinese word vector
# Download word vectors file from https://www.kaggle.com/kerneler/starter-sgns-merge-word-18e5b7b5-9/execution?select=sgns.merge.word
# file to a folder with this jupiter notebook (size 3.5Gb)
word_to_vec_zh = read_glove_vecs('sgns.merge.word')

In [272]:
# We could not find glove nodel for finnish,
# so will work with finnish language separately

In [512]:
# Create column for calculating cosine similarity
scores_cs['Cosine_similarity'], scores_de['Cosine_similarity'] = 0, 0
scores_ru['Cosine_similarity'], scores_zh['Cosine_similarity'] = 0, 0
scores_en_fi['Cosine_similarity'], scores_en_zh['Cosine_similarity'] = 0, 0

In [513]:
# Check cosine similarity between two sentences (reference and translation)
# Use different pre-trained word embeddings:
# GloVe - for cs, de, ru, zh that translated to english
# 'sgns.merge.word' - for en-zh pair
def cos_s(ref_list, trans_list, scores):
    for doc in range(len(scores)):
        ref_embeddings = np.zeros((len(scores[doc]),300))
        tr_embeddings = np.zeros((len(scores[doc]),300))

        norms_doc_embeddings_tr = np.zeros((len(trans_list[doc]),1))
        norms_doc_embeddings_ref = np.zeros((len(ref_list[doc]),1))

        Cosine_similarity = []

        for i in tqdm(range(len(trans_list[doc]))):
            if doc == 4:
                words_freq = words_count_zh(trans_list[doc][i])
                doc_embeddings = np.zeros(word_to_vec_zh["a"].shape)

            else:
                words_freq = words_count(trans_list[doc][i])
                doc_embeddings = np.zeros(word_to_vec_map["a"].shape)
            num_words = 0
            for word_freq in words_freq:
                word = word_freq[0]
                freq = word_freq[1]
                try:
                    #adding word embeddings for each word in the document
                    if doc == 4:
                        doc_embeddings += (word_to_vec_zh[word] * freq)
                    else:
                        doc_embeddings += (word_to_vec_map[word] * freq)
                    num_words += freq
                except:
                    continue
            try:
                # doing average
                doc_embeddings /= num_words
            except:
                print("divide by zero encountered for article at index "+str(i))
                continue
            norms_doc_embeddings_tr[i,:] = np.sqrt(np.dot(doc_embeddings,doc_embeddings))
            tr_embeddings[i,:] = doc_embeddings


            if doc == 4:
                words_freq = words_count_zh(ref_list[doc][i])
                doc_embeddings = np.zeros(word_to_vec_zh["a"].shape)
            else:
                words_freq = words_count(ref_list[doc][i])
                doc_embeddings = np.zeros(word_to_vec_map["a"].shape)
            num_words = 0
            for word_freq in words_freq:
                word = word_freq[0]
                freq = word_freq[1]
                try:
                    #adding word embeddings for each word in the document
                    if doc == 4:
                        doc_embeddings += (word_to_vec_zh[word] * freq)

                    else:
                        doc_embeddings += (word_to_vec_map[word] * freq)
                    num_words += freq
                except:
                    continue
            try:
                # doing average
                doc_embeddings /= num_words
            except:
                print("divide by zero encountered for article at index "+str(i))
                continue
            norms_doc_embeddings_ref[i,:] = np.sqrt(np.dot(doc_embeddings,doc_embeddings))
            ref_embeddings[i,:] = doc_embeddings

            # Calculate cosine similarity using dot product of two vectors
            # and norms of vectors

            #  The cosine similarity depends on the angle between  vectors .
            #  If  vectors  are very similar, their cosine similarity will be close to 1
            norm_prods = norms_doc_embeddings_ref[i] * norms_doc_embeddings_tr[i]
            dot_prod = np.dot(ref_embeddings[i], tr_embeddings[i].reshape(300,1))
            cos_similarity = dot_prod / norm_prods

            Cosine_similarity.append(float(cos_similarity))

        scores[doc]['Cosine_similarity'] = pd.Series(Cosine_similarity)

In [514]:
# Create lists with languages to work with
ref_list = [references_cs10, references_de10, references_ru10, references_zh10, references_en_zh10]
trans_list = [translation_cs10, translation_de10, translation_ru10, translation_zh10, translation_en_zh10]
scores = [scores_cs, scores_de, scores_ru, scores_zh, scores_en_zh]
# Calculate cosine similarity 
cos_s(ref_list, trans_list, scores) 

In [515]:
def corr_new(scores):
    
    cos = scores['z-score'].corr(scores['Cosine_similarity'])
    return cos

In [516]:
corr_cs_new = corr_new(scores_cs)
corr_de_new = corr_new(scores_de)
corr_ru_new = corr_new(scores_ru)
corr_zh_new = corr_new(scores_zh)

corr_en_zh_new = corr_new(scores_en_zh)

In [517]:
index = ['Cosine_similarity (w/o stopwords)']
data = []
columns = ['scores_cs', 'scores_de',  'scores_en_zh', 'scores_ru', 'scores_zh']
correlation = pd.DataFrame(data, columns=columns, index = index)
correlation.iloc[:,0] = corr_cs_new
correlation.iloc[:,1] = corr_de_new
correlation.iloc[:,2] = corr_en_zh_new

correlation.iloc[:,3] = corr_ru_new
correlation.iloc[:,4] = corr_zh_new

In [518]:
# Calculate correlation between z-score and cosine_similarity columns
# for all language pairs except finnish
correlation

,scores_cs,scores_de,scores_en_zh,scores_ru,scores_zh
Cosine_similarity (w/o stopwords),0.368199,0.290665,0.396741,0.304872,0.271934


In [486]:
# Create lists with pre-processed series to work with
# All parameters of pre-processing set to True except Stemmer
ref_list = [references_cs11, references_de11, references_ru11, references_zh11, references_en_zh11]
trans_list = [translation_cs11, translation_de11, translation_ru11, translation_zh11, translation_en_zh11]
scores = [scores_cs, scores_de, scores_ru, scores_zh, scores_en_zh]
# Calculate cosine similarity (without lemmatizationm removing stop-words and )
cos_s(ref_list, trans_list, scores) 

In [487]:
corr_cs_new = corr_new(scores_cs)
corr_de_new = corr_new(scores_de)
corr_ru_new = corr_new(scores_ru)
corr_zh_new = corr_new(scores_zh)
corr_en_zh_new = corr_new(scores_en_zh)
index = ['Cosine_similarity (pre-processing)']
data = []
columns = ['scores_cs', 'scores_de',  'scores_en_zh', 'scores_ru', 'scores_zh']
correlation = pd.DataFrame(data, columns=columns, index = index)
correlation.iloc[:,0] = corr_cs_new
correlation.iloc[:,1] = corr_de_new
correlation.iloc[:,2] = corr_en_zh_new

correlation.iloc[:,3] = corr_ru_new
correlation.iloc[:,4] = corr_zh_new

In [488]:
# Calculate correlation between z-score and cosine_similarity columns
# for all language pairs except finnish 
# All parameters of pre-processing set to True except Stemmer
correlation

,scores_cs,scores_de,scores_en_zh,scores_ru,scores_zh
Cosine_similarity (pre-processing),0.360295,0.287918,0.396771,0.307792,0.256307


### Predict for test-set (except Finnish)

In [527]:
scores_cs_test = pd.read_csv("testset/cs-en/scores.csv")
scores_de_test = pd.read_csv("testset/de-en/scores.csv")
scores_en_fi_test = pd.read_csv("testset/en-fi/scores.csv")
scores_en_zh_test = pd.read_csv("testset/en-zh/scores.csv")
scores_ru_test = pd.read_csv("testset/ru-en/scores.csv")
scores_zh_test = pd.read_csv("testset/zh-en/scores.csv")

In [528]:
def prep_test(ref_trans, pontuation = True, lemmatize=True, stemmer=True, stop_words = True):
    
    if ref_trans == 'referece':
        a,b,c = scores_cs_test['reference'], scores_de_test['reference'], scores_en_fi_test['reference']
        d,e,f = scores_en_zh_test['reference'], scores_ru_test['reference'].astype(str), scores_zh_test['reference']
    elif ref_trans == 'translation':
        a,b,c = scores_cs_test['translation'], scores_de_test['translation'], scores_en_fi_test['translation']
        d,e,f = scores_en_zh_test['translation'], scores_ru_test['translation'].astype(str), scores_zh_test['translation']
        
    ref_trans_cs = preprocessing(a,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_de = preprocessing(b,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_en_fi = preprocessing(c,'finnish', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_en_zh = preprocessing(d,'zh', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_ru = preprocessing(e,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    ref_trans_zh = preprocessing(f,'eng', pontuation = pontuation, 
                                  lemmatize=lemmatize, stemmer=stemmer, stop_words = stop_words)
    
    out = ref_trans_cs, ref_trans_de, ref_trans_en_fi, ref_trans_en_zh, ref_trans_ru, ref_trans_zh
    
    return out

In [529]:
# Create column for calculating cosine similarity
scores_cs_test['Cosine_similarity'], scores_de_test['Cosine_similarity'] = 0, 0
scores_ru_test['Cosine_similarity'], scores_zh_test['Cosine_similarity'] = 0, 0
scores_en_fi_test['Cosine_similarity'], scores_en_zh_test['Cosine_similarity'] = 0, 0

In [530]:
# All parameters set to False
# (stopwords and punctuation will be deleted by words_count functions)
references_cs_test, references_de_test, references_en_fi_test, references_en_zh_test, references_ru_test, references_zh_test = prep_test(
    'referece', pontuation = False, lemmatize=False, stemmer=False, stop_words = False)

translation_cs_test, translation_de_test, translation_en_fi_test, translation_en_zh_test, translation_ru_test, translation_zh_test = prep_test(
    'translation', pontuation = False, lemmatize=False, stemmer=False, stop_words = False)

In [531]:
# Create lists with pre-processed series to work with
# All parameters of pre-processing set to True except Stemmer
ref_list_test = [references_cs_test, references_de_test, references_ru_test, references_zh_test, references_en_zh_test]
trans_list_test = [translation_cs_test, translation_de_test, translation_ru_test, translation_zh_test, translation_en_zh_test]
scores_test = [scores_cs_test, scores_de_test, scores_ru_test, scores_zh_test, scores_en_zh_test]
# Calculate cosine similarity (without lemmatizationm removing stop-words and )
cos_s(ref_list_test, trans_list_test, scores_test) 

In [532]:
for doc in scores_test:
    doc = doc.rename(columns={'Cosine_similarity': 'metric'}, inplace=True)


In [534]:
# Save to csv file with 'metric' column


scores_cs_test.to_csv("testset/cs-en/scores.csv", index=False)
scores_de_test.to_csv("testset/de-en/scores.csv", index=False)

scores_en_zh_test.to_csv("testset/en-zh/scores.csv", index=False)
scores_ru_test.to_csv("testset/ru-en/scores.csv", index=False)
scores_zh_test.to_csv("testset/zh-en/scores.csv", index=False)

### Fiinish language:
- use multilingual BERT and finBERT to get sentence embeddings

##### BERT

In [538]:
# Download pre-trained multilingual BERT model 
# to get sentence embeddings
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained("bert-base-multilingual-cased")

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [539]:
# Check if everything works ok
text = "Long text about stars."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
last_hidden_states = output[0]
cls_embedding = last_hidden_states[0][0]  

In [540]:
# Check cosine similarity for identical lines - should be equal 1
normalize_a = tf.nn.l2_normalize(cls_embedding,0)        
normalize_b = tf.nn.l2_normalize(cls_embedding,0)
cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
float(cos_similarity)

1.0000001192092896

In [541]:
# Attention: this cell takes more than hour to calculate cosine similaritites

# Compare the embeddings of sentences that we receive with  pretrained BERT 
scores_en_fi['Cosine_similarity_base'] = 0
Cosine = []
for doc in tqdm(range(len(scores_en_fi))):
    
    # text from references_en_fi and translation_en_fi
    text1 = references_en_fi1[doc]
    text2 = translation_en_fi1[doc]
    
    encoded_input1 = tokenizer(text1, return_tensors='tf')
    output1 = model(encoded_input1)
    
    encoded_input2 = tokenizer(text2, return_tensors='tf')
    output2 = model(encoded_input2)
    
    last_hidden_states_1 = output1[0]
    cls_embedding_1 = last_hidden_states_1[0][0]   
    last_hidden_states_2 = output2[0]
    cls_embedding_2 = last_hidden_states_2[0][0]
    normalize_a = tf.nn.l2_normalize(cls_embedding_1,0)        
    normalize_b = tf.nn.l2_normalize(cls_embedding_2,0)
    cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
    Cosine.append(float(cos_similarity))
    
scores_en_fi['Cosine_similarity_base'] = pd.Series(Cosine)

In [542]:
# Calculate Pearson correlation
scores_en_fi['z-score'].corr(scores_en_fi['Cosine_similarity_base'])

0.15062687657592824

In [48]:
# We will pre-process finish docs, trying to increase our correlation with z-score

In [543]:
# Attention: this library can have some isuues during installation
# Some advices how to resolve them from offical page - https://voikko.puimula.org/python.html

# Check how this library works:
#Define a Voikko class for Finnish
v = libvoikko.Voikko(u"fi")

#A word that might or might not be in base form
word = "lehmähauteen"

#Analyze the word
voikko_dict = v.analyze(word)

#Extract the base form as
#analyze() function returns various info for the word
word_baseform = voikko_dict[0]['BASEFORM']

#Print the base form of the word
print(word_baseform)

lehmähaude


In [552]:
# delete stopwords, lemmatize using Voikko
def preprocess_fi(alltext):
    processed_corpus = []
    #stop_words = set(stopwords.words())
    for i in tqdm(range(len(alltext))):
        text = alltext[i]
        # Tokenize
        tokenizer = RegexpTokenizer(r'\w+')
        # tokenize text
        tokens = tokenizer.tokenize(text)
        temp = []
        #Define a Voikko class for Finnish
        v = libvoikko.Voikko(u"fi")

        # Change words to lemmas
        for word in tokens:
            voikko_dict = v.analyze(word)
            try:
                t = voikko_dict[0]['BASEFORM']
                temp.append(t)
            except:
                temp.append(word)
                continue
        tokens = temp
        # delete stop words
        tokens = list(filter(lambda x: x not in stop_words_fi, tokens))
        #Convert to lowercase because Voikko returns some words that begin from upper case
        text = text.lower()
        text = " ".join(tokens)
        processed_corpus.append(text)
    return processed_corpus

In [553]:
# Pre-process finnish sentences (references and translations using preprocess_fi)
references_en_fi_new = preprocess_fi(references_en_fi1)
translation_en_fi_new = preprocess_fi(translation_en_fi1)

In [554]:
# Download pre-trained multilingual BERT model 
# to get sentence embeddings for pre-processed references and translations

tokenizer1 = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model1 = TFBertModel.from_pretrained("bert-base-multilingual-cased")

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [555]:
# Attention: this cell takes more than hour to calculate cosine similaritites

# Compare the embeddings of sentences that we receive with  pretrained BERT 
# for pre-processed finnish text (we pre-processed it in a 1st part) 
# (cleaned andd lemmatized with Voikko)
scores_en_fi['Cosine_similarity'] = 0
Cosine = []
for doc in tqdm(range(len(scores_en_fi))):
    
    # text from references_en_fi and translation_en_fi
    text1 = references_en_fi_new[doc]
    text2 = translation_en_fi_new[doc]
    
    encoded_input1 = tokenizer1(text1, return_tensors='tf')
    output1 = model1(encoded_input1)
    
    encoded_input2 = tokenizer1(text2, return_tensors='tf')
    output2 = model1(encoded_input2)
    
    last_hidden_states_1 = output1[0]
    cls_embedding_1 = last_hidden_states_1[0][0]   
    last_hidden_states_2 = output2[0]
    cls_embedding_2 = last_hidden_states_2[0][0]
    normalize_a = tf.nn.l2_normalize(cls_embedding_1,0)        
    normalize_b = tf.nn.l2_normalize(cls_embedding_2,0)
    cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
    Cosine.append(float(cos_similarity))
    
scores_en_fi['Cosine_similarity'] = pd.Series(Cosine)

In [556]:
# Calculate Pearson correlation
scores_en_fi['z-score'].corr(scores_en_fi['Cosine_similarity'])

0.1615293553059098

In [ ]:
# We increased to 0.161 from 0.150, but still a room for improvment - we will try to work with finBERT
# BERT model that is trained and fine-tuned for finnish

#### finBERT from TurkuNLP

In [557]:
# Download pre-trained finnish language BERT model - finBERT from TurkuNLP 
# to get sentence embeddings

tokenizer2 = BertTokenizer.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')
model2 = TFBertModel.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

Some layers from the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [558]:
# Checking if our model works - cakculate cosine simila
text = "varastettujen polkupyörän ostajat ovat tavanomaisia suomalaisia, poliisin mukaan."
encoded_input = tokenizer2(text, return_tensors='tf')
output = model2(encoded_input)
last_hidden_states = output[0]
cls_embedding = last_hidden_states[0][0]  
normalize_a = tf.nn.l2_normalize(cls_embedding,0)        
normalize_b = tf.nn.l2_normalize(cls_embedding,0)
cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
float(cos_similarity)


1.000000238418579

In [559]:
# Attention: this cell takes more than hour to calculate cosine similaritites

# Compare the embeddings of sentences that we receive with  pretrained finBERT 
# (without lemmatization)

scores_en_fi['Cosine_similarity_finBERT_base'] = 0
Cosine = []
for doc in tqdm(range(len(scores_en_fi))):
    
    # text from references_en_fi and translation_en_fi
    text1 = references_en_fi1[doc]
    text2 = translation_en_fi1[doc]
    
    encoded_input1 = tokenizer2(text1, return_tensors='tf')
    output1 = model2(encoded_input1)
    
    encoded_input2 = tokenizer2(text2, return_tensors='tf')
    output2 = model2(encoded_input2)
    
    last_hidden_states_1 = output1[0]
    cls_embedding_1 = last_hidden_states_1[0][0]   
    last_hidden_states_2 = output2[0]
    cls_embedding_2 = last_hidden_states_2[0][0]
    normalize_a = tf.nn.l2_normalize(cls_embedding_1,0)        
    normalize_b = tf.nn.l2_normalize(cls_embedding_2,0)
    cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
    Cosine.append(float(cos_similarity))
    
scores_en_fi['Cosine_similarity_finBERT_base'] = pd.Series(Cosine)

In [560]:
# Print cosine similarity
scores_en_fi['z-score'].corr(scores_en_fi['Cosine_similarity_finBERT_base'])

0.6164684269163455

In [ ]:
# finBERT with pre-processed data

In [561]:
# Download pre-trained finnish language BERT model - finBERT from TurkuNLP 
# to get sentence embeddings

tokenizer3 = BertTokenizer.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')
model3 = TFBertModel.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

Some layers from the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [562]:
# Attention: this cell takes more than hour to calculate cosine similaritites

# Compare the embeddings of sentences that we receive with  pretrained finBERT 
# and pre-processed finnish text - cleaned and lemmatized

scores_en_fi['Cosine_similarity_finBERT'] = 0
Cosine = []
for doc in tqdm(range(len(scores_en_fi))):
    
    # text from references_en_fi and translation_en_fi
    text1 = references_en_fi_new[doc]
    text2 = translation_en_fi_new[doc]
    
    encoded_input1 = tokenizer3(text1, return_tensors='tf')
    output1 = model3(encoded_input1)
    
    encoded_input2 = tokenizer3(text2, return_tensors='tf')
    output2 = model3(encoded_input2)
    
    last_hidden_states_1 = output1[0]
    cls_embedding_1 = last_hidden_states_1[0][0]   
    last_hidden_states_2 = output2[0]
    cls_embedding_2 = last_hidden_states_2[0][0]
    normalize_a = tf.nn.l2_normalize(cls_embedding_1,0)        
    normalize_b = tf.nn.l2_normalize(cls_embedding_2,0)
    cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
    Cosine.append(float(cos_similarity))
    
scores_en_fi['Cosine_similarity_finBERT'] = pd.Series(Cosine)

In [563]:
# Print cosine similarity
scores_en_fi['z-score'].corr(scores_en_fi['Cosine_similarity_finBERT'])

0.4842557556811352

In [566]:
# Compare cosine similarities for all models
index = ['Multilingual BERT without pre-process', 'Multilingual BERT with pre-process', 
           'finBERT (TurkuNLP) without pre-process', 'finBERT (TurkuNLP) with pre-process']
data = []
columns = ['en-fi']
finish_corr = pd.DataFrame(data, columns=columns, index = index)
finish_corr.iloc[0] = scores_en_fi['z-score'].corr(scores_en_fi['Cosine_similarity_base'])
finish_corr.iloc[1] = scores_en_fi['z-score'].corr(scores_en_fi['Cosine_similarity'])
finish_corr.iloc[2] = scores_en_fi['z-score'].corr(scores_en_fi['Cosine_similarity_finBERT_base'])
finish_corr.iloc[3] = scores_en_fi['z-score'].corr(scores_en_fi['Cosine_similarity_finBERT'])

In [567]:
finish_corr

,en-fi
Multilingual BERT without pre-process,0.150627
Multilingual BERT with pre-process,0.161529
finBERT (TurkuNLP) without pre-process,0.616468
finBERT (TurkuNLP) with pre-process,0.484256


### Calculate metric for test set en-fi

In [569]:
# Attention: this cell takes more than hour to calculate cosine similaritites

# Compare the embeddings of sentences that we receive with  pretrained finBERT 
# (without lemmatization)

scores_en_fi_test['Cosine_similarity'] = 0
Cosine = []
for doc in tqdm(range(len(scores_en_fi_test))):
    
    # text from references_en_fi and translation_en_fi
    text1 = references_en_fi_test[doc]
    text2 = translation_en_fi_test[doc]
    
    encoded_input1 = tokenizer2(text1, return_tensors='tf')
    output1 = model2(encoded_input1)
    
    encoded_input2 = tokenizer2(text2, return_tensors='tf')
    output2 = model2(encoded_input2)
    
    last_hidden_states_1 = output1[0]
    cls_embedding_1 = last_hidden_states_1[0][0]   
    last_hidden_states_2 = output2[0]
    cls_embedding_2 = last_hidden_states_2[0][0]
    normalize_a = tf.nn.l2_normalize(cls_embedding_1,0)        
    normalize_b = tf.nn.l2_normalize(cls_embedding_2,0)
    cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
    Cosine.append(float(cos_similarity))
    
scores_en_fi_test['Cosine_similarity'] = pd.Series(Cosine)

In [570]:
scores_en_fi_test.rename(columns={'Cosine_similarity': 'metric'}, inplace=True)

In [571]:
scores_en_fi_test

,source,reference,translation,metric
0,One local resident who did not wish to be name...,"Eräs paikallinen asukas, joka ei halunnut nime...",Toisen nimettömänä pysyttelevän asukkaan mukaa...,0.754227
1,"Still, she clings to a chant she's committed t...",Silti hän takertuu chant hän on sitoutunut mui...,"Silti hän luottaa edelleen iskulauseeseen, jon...",0.735724
2,"I don't want to be asked, 'What were you doing...","En halua, että minulta kysytään: ""Mitä te teit...","En halua, että kenenkään tarvitsee kysyä minul...",0.868746
3,"""I wouldn't say it was a lie – that's a pretty...","""En sanoisi, että se oli valhe - se on aika ro...","En sanoisi, että se oli valhe, se on aika kova...",0.855505
4,Kari Kola took part in the opening ceremony of...,Kari Kola osallistui valon vuoden avajaisiin v...,Kari Kola oli mukana Valon teemavuoden avajais...,0.975093
...,...,...,...,...
8092,Dubai's civil defence authorities said firefig...,Dubain siviilipuolustusviranomaisten mukaan pa...,Dubain pelastusviranomaisten mukaan sammutusjo...,0.937237
8093,But most contemporary animals that have counte...,"Mutta useimmat nykyajan eläimet, jotka ovat va...","Nykyisistä eläimistä esimerkiksi peuroilla, se...",0.854175
8094,"Tommi, his spouse Riina and their two small ch...","Tommi, hänen puolisonsa Riina ja heidän kaksi ...","Tommi, hänen avopuolisonsa Riina ja pariskunna...",0.973322
8095,Tenacious outdoor swim school students in Imat...,Sitkeitä ulko-uintikoululaisia Imatralla ei hu...,Imatran sinnikkäitä rantauimakoululaisia ei he...,0.914061


In [572]:
scores_en_fi_test.to_csv("testset/en-fi/scores.csv", index=False)